<a href="https://colab.research.google.com/github/wlsgud623/vaccine_tweet_analysis/blob/main/VaccineTweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
!pip install transformers==3

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import logging

logging.basicConfig(level=logging.ERROR)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

     |████████████████████████████████| 754 kB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 35.6 MB/s 
     |████████████████████████████████| 895 kB 56.8 MB/s 
     |████████████████████████████████| 3.0 MB 36.5 MB/s 
cpu


# Load Data

Kaggle API

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jinhyeoungbae","key":"dc42a77071a666d47d2e97f076322dfb"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d kaushiksuresh147/covidvaccine-tweets
! kaggle datasets download -d gpreda/all-covid19-vaccines-tweets
! kaggle datasets download -d cosmos98/twitter-and-reddit-sentimental-analysis-dataset

 77% 41.0M/53.1M [00:00<00:00, 74.5MB/s]
100% 53.1M/53.1M [00:00<00:00, 108MB/s] 
 64% 19.0M/29.9M [00:00<00:00, 52.3MB/s]
100% 29.9M/29.9M [00:00<00:00, 99.6MB/s]
 90% 9.00M/10.0M [00:00<00:00, 92.9MB/s]
100% 10.0M/10.0M [00:00<00:00, 91.4MB/s]


In [ ]:
!unzip all-covid19-vaccines-tweets.zip
!unzip covidvaccine-tweets.zip
!unzip twitter-and-reddit-sentimental-analysis-dataset.zip

Archive:  all-covid19-vaccines-tweets.zip
replace vaccination_all_tweets.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  covidvaccine-tweets.zip
replace covidvaccine.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  twitter-and-reddit-sentimental-analysis-dataset.zip
  inflating: Reddit_Data.csv         
  inflating: Twitter_Data.csv        


Kaggle Tweet Data

In [ ]:
first_vaccine_data = pd.read_csv("/content/covidvaccine.csv", nrows = 1000)
second_vaccine_data = pd.read_csv("/content/vaccination_all_tweets.csv", nrows = 1000)

In [ ]:
first_vaccine_data.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64.0,11.0,110.0,False,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False
1,Shubham Gupta,NaN,I will tell about all experiences of my life f...,14-08-2020 16:42,1.0,17.0,0.0,False,18-08-2020 12:55,#CoronavirusVaccine #CoronaVaccine #CovidVacci...,"['CoronavirusVaccine', 'CoronaVaccine', 'Covid...",Twitter for Android,False
2,Journal of Infectiology,NaN,Journal of Infectiology (ISSN 2689-9981) is ac...,14-12-2017 07:07,143.0,566.0,8.0,False,18-08-2020 12:46,Deaths due to COVID-19 in Affected Countries\n...,NaN,Twitter Web App,False
3,Zane,NaN,Fresher than you.,18-09-2019 11:01,29.0,25.0,620.0,False,18-08-2020 12:45,@Team_Subhashree @subhashreesotwe @iamrajchoco...,NaN,Twitter for Android,False
4,Ann-Maree O’Connor,"Adelaide, South Australia",Retired university administrator. Melburnian b...,24-01-2013 14:53,83.0,497.0,10737.0,False,18-08-2020 12:45,@michellegrattan @ConversationEDU This is what...,NaN,Twitter Web App,False


In [ ]:
second_vaccine_data.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [ ]:
data_column = ['user_name', 'user_description','user_location','user_followers','text']
first_vaccine_data = first_vaccine_data[data_column]
second_vaccine_data = second_vaccine_data[data_column]

In [ ]:
print(first_vaccine_data['text'])

0      Australia to Manufacture Covid-19 Vaccine and ...
1      #CoronavirusVaccine #CoronaVaccine #CovidVacci...
2      Deaths due to COVID-19 in Affected Countries\n...
3      @Team_Subhashree @subhashreesotwe @iamrajchoco...
4      @michellegrattan @ConversationEDU This is what...
                             ...                        
995    Phase I/II data on Pfizer/BioNTech's mRNA #Cov...
996    RUSSIA'S COVID VACCINE: SHOULD WE BE EXCITED?\...
997    @anandmahindra please help us spread the word ...
998    @tehseenp please help us spread the word #covi...
999    @zeeshan_iyc please help us spread the word #c...
Name: text, Length: 1000, dtype: object


In [ ]:
import re

def con(x):
  return ' '.join(re.sub("(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
    
first_vaccine_data['text']=first_vaccine_data['text'].apply(con)
second_vaccine_data['text']=second_vaccine_data['text'].apply(con)

In [ ]:
print(first_vaccine_data['text'])
print(first_vaccine_data['text'][0])

0      Australia to Manufacture Covid 19 Vaccine and ...
1                           Australia is doing very good
2      Deaths due to COVID 19 in Affected Countries R...
3                         Subhashree Stay safe di amp da
4      This is what passes for leadership in our coun...
                             ...                        
995    Phase I II data on Pfizer BioNTech s mRNA It i...
996    RUSSIA S COVID VACCINE SHOULD WE BE EXCITED Is...
997                       please help us spread the word
998                       please help us spread the word
999                   iyc please help us spread the word
Name: text, Length: 1000, dtype: object
Australia to Manufacture Covid 19 Vaccine and give it to the Citizens for free of cost AFP quotes Prime Minister


Train Data

In [ ]:
#(0 = negative,  = neutral, 4 = positive)
test_data = pd.read_csv("/content/Twitter_Data.csv", nrows=20000)
test_data.head()


,clean_text,category
0,when modi promised “minimum government maximum...,-1
1,talk all the nonsense and continue all the dra...,0
2,what did just say vote for modi welcome bjp t...,1
3,asking his supporters prefix chowkidar their n...,1
4,answer who among these the most powerful world...,1


# Pre-processing

In [ ]:
#Tokenize Data
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

# Text DataSet

In [ ]:
class GPReviewDataset(Dataset):
  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.reviews)
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
# 최대 토큰 길이
MAX_LEN = 100

In [ ]:
df_train, df_test = train_test_split(
  test_data,
  test_size=0.1,
  random_state=RANDOM_SEED
)

df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=RANDOM_SEED
)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(9282, 3)
(516, 3)
(516, 3)


In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df['message to examine'].to_numpy(),
    targets=df['label (depression result)'].to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers = 2
  )

In [ ]:
print(df_train.columns)

Index(['Index', 'message to examine', 'label (depression result)'], dtype='object')


In [ ]:
BATCH_SIZE = 16

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

# Import Model

In [ ]:
bert_model = BertModel.from_pretrained('bert-base-cased')

In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
class_names = ['negative', 'neutral', 'positive']

model = SentimentClassifier(len(class_names))
model = model.to(device)

In [ ]:
data = next(iter(train_data_loader))

input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length
print(input_ids[0])
print()

torch.Size([16, 100])
torch.Size([16, 100])
tensor([  101,  5946,  1770,   117,  3455,   176,  8209, 23311,  7376,   117,
         1105,  5578,  1106,  9419,  1104,  5837,  1320,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       device='cuda:0')



In [ ]:
F.softmax(model(input_ids, attention_mask), dim=1)

tensor([[0.4269, 0.2447, 0.3283],
        [0.2522, 0.4868, 0.2610],
        [0.5175, 0.1970, 0.2855],
        [0.2702, 0.3611, 0.3687],
        [0.5554, 0.2342, 0.2104],
        [0.3923, 0.2499, 0.3578],
        [0.4756, 0.2690, 0.2554],
        [0.3879, 0.3869, 0.2252],
        [0.3173, 0.3715, 0.3112],
        [0.4059, 0.3862, 0.2080],
        [0.3085, 0.3773, 0.3143],
        [0.3810, 0.3948, 0.2242],
        [0.4157, 0.3032, 0.2812],
        [0.2479, 0.3826, 0.3695],
        [0.3779, 0.1632, 0.4589],
        [0.3905, 0.2542, 0.3553]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

# Train

In [ ]:
EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_examples
):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)
      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')
  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
  )
  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/10
----------
Train loss 0.04080355026488306 accuracy 0.9886877828054298
Val   loss 8.693254582767347e-05 accuracy 1.0

Epoch 2/10
----------
Train loss 0.006658649665498689 accuracy 0.9984917043740573
Val   loss 3.597117844919646e-05 accuracy 1.0

Epoch 3/10
----------
Train loss 0.0010343169491653325 accuracy 0.9995690583925878
Val   loss 1.966967338927189e-05 accuracy 1.0

Epoch 4/10
----------
Train loss 2.4662082198899172e-05 accuracy 1.0
Val   loss 1.3334337795227343e-05 accuracy 1.0

Epoch 5/10
----------
Train loss 1.776987082128709e-05 accuracy 1.0
Val   loss 1.0177535152637795e-05 accuracy 1.0

Epoch 6/10
----------
Train loss 1.3838409288425585e-05 accuracy 1.0
Val   loss 8.210215927454446e-06 accuracy 1.0

Epoch 7/10
----------
Train loss 1.1738744370771368e-05 accuracy 1.0
Val   loss 6.980453359905038e-06 accuracy 1.0

Epoch 8/10
----------
Train loss 9.964855979826894e-06 accuracy 1.0
Val   loss 6.196803193840121e-06 accuracy 1.0

Epoch 9/10
----------
Train loss 

# Test Raw Text

In [ ]:
example = first_vaccine_data["text"][0:10]
for text in example:
  print("text: " +text)

text: Australia to Manufacture Covid-19 Vaccine and give it to the Citizens for free of cost: AFP quotes Prime Minister
#CovidVaccine
text: #CoronavirusVaccine #CoronaVaccine #CovidVaccine Australia is doing very good https://t.co/kBT7l6pArY
text: Deaths due to COVID-19 in Affected Countries
Read More: https://t.co/V8Y3Stu0UW
@r_piryani @shitalbhandary… https://t.co/6jpMxX2KtI
text: @Team_Subhashree @subhashreesotwe @iamrajchoco Stay safe @subhashreesotwe di &amp; @iamrajchoco da ❤️❤️… https://t.co/ayhoaQm0Ls
text: @michellegrattan @ConversationEDU This is what passes for leadership in our country: a voucher for something that w… https://t.co/OUUb1PeYIj
text: The Multi-system Inflammatory Syndrome-Children (MIS-C) w/ #COVID19 (atypical Kawasaki disease) #COVID19India 

The… https://t.co/98wj2CEkez
text: @PrivilRodrigues @yatish57 @deepkaranahuja @shristi522 @Amrita33392520 @RashmiSriniva14 @AkashRK_88 @SJanaQA… https://t.co/VPLa11fwfr
text: @MSNBC Well, let’s qualify that: would anyone

In [ ]:
encoded_example = []

for data in example:
  encoded_tweet = tokenizer.encode_plus(
  data,
  max_length=MAX_LEN,
  add_special_tokens=True,
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',
)
  encoded_example.append(encoded_tweet)

In [ ]:
for encoded_data in encoded_example:
  input_ids = encoded_data['input_ids'].to(device)
  attention_mask = encoded_data['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)

  print(prediction)


tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([1], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([0], device='cuda:0')
tensor([1], device='cuda:0')
tensor([0], device='cuda:0')
